In [ ]:
# Analýza Blog Authorship Corpus - Extrakce dolarových částek
# Dataset: https://www.kaggle.com/datasets/rtatman/blog-authorship-corpus

import pandas as pd
import re
import numpy as np
from typing import List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Načtení datasetu
print("Načítání datasetu...")
try:
    # Načtení CSV souboru
    df = pd.read_csv('blogtext.csv', encoding='utf-8')
    print(f"Dataset úspěšně načten. Počet řádků: {len(df)}")
    print(f"Sloupce: {list(df.columns)}")
except FileNotFoundError:
    print("Soubor 'blogtext.csv' nebyl nalezen.")
    print("Ujistěte se, že máte stažený dataset z Kaggle a soubor je ve stejné složce jako tento notebook.")
    exit()

# Zobrazení základních informací o datasetu
print("\nZákladní informace o datasetu:")
print(df.info())
print("\nPrvních 5 řádků:")
print(df.head())

def extract_dollar_amounts(text: str) -> List[float]:
    """
    Extrahuje všechny dolarové částky z textu.
    
    Args:
        text (str): Text ke zpracování
    
    Returns:
        List[float]: Seznam nalezených částek
    """
    if pd.isna(text) or not isinstance(text, str):
        return []
    
    # Regex pattern pro nalezení dolarových částek
    # Pokrývá formáty jako: $123, $1,234, $1.23, $1,234.56, $12.34K, $1.2M, $1B
    patterns = [
        r'\$\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s*[BMK]?',  # $1,234.56, $123K, $1.2M, $1B
        r'\$\s*(\d+(?:\.\d{1,2})?)\s*(?:billion|million|thousand|k|m|b)',  # $123 million
        r'\$\s*(\d+(?:,\d{3})*(?:\.\d{1,2})?)',  # základní formát $1,234.56
    ]
    
    amounts = []
    
    for pattern in patterns:
        matches = re.findall(pattern, text.lower(), re.IGNORECASE)
        for match in matches:
            try:
                # Odstranění čárek z čísla
                clean_amount = match.replace(',', '')
                amount = float(clean_amount)
                
                # Zpracování jednotek (K, M, B)
                if 'k' in text.lower() or 'thousand' in text.lower():
                    amount *= 1000
                elif 'm' in text.lower() or 'million' in text.lower():
                    amount *= 1000000
                elif 'b' in text.lower() or 'billion' in text.lower():
                    amount *= 1000000000
                
                amounts.append(amount)
            except ValueError:
                continue
    
    return amounts

def find_dollar_amounts_advanced(text: str) -> List[float]:
    """
    Pokročilá funkce pro hledání dolarových částek s různými formáty.
    """
    if pd.isna(text) or not isinstance(text, str):
        return []
    
    amounts = []
    
    # Pattern 1: Základní $123.45 formát
    pattern1 = r'\$\s*(\d{1,3}(?:,\d{3})*(?:\.\d{1,2})?)'
    matches1 = re.findall(pattern1, text)
    for match in matches1:
        try:
            amount = float(match.replace(',', ''))
            amounts.append(amount)
        except ValueError:
            continue
    
    # Pattern 2: $123K, $1.2M, $5B formáty
    pattern2 = r'\$\s*(\d+(?:\.\d+)?)\s*([KMB])'
    matches2 = re.findall(pattern2, text, re.IGNORECASE)
    for amount_str, unit in matches2:
        try:
            amount = float(amount_str)
            if unit.upper() == 'K':
                amount *= 1000
            elif unit.upper() == 'M':
                amount *= 1000000
            elif unit.upper() == 'B':
                amount *= 1000000000
            amounts.append(amount)
        except ValueError:
            continue
    
    return amounts

# Analýza textu pro hledání dolarových částek
print("\nHledání dolarových částek v textech...")

# Předpokládáme, že text je ve sloupci 'text' - upravte podle skutečné struktury
text_column = None
for col in df.columns:
    if 'text' in col.lower() or 'blog' in col.lower() or 'content' in col.lower():
        text_column = col
        break

if text_column is None:
    print("Nebyl nalezen sloupec s texty. Dostupné sloupce:")
    for col in df.columns:
        print(f"- {col}")
    # Použijeme první sloupec, který vypadá jako text
    text_column = df.columns[-1]  # obvykle je text v posledním sloupci
    print(f"Používám sloupec: {text_column}")

print(f"Analyzuji sloupec: {text_column}")

# Extrakce všech dolarových částek
all_amounts = []
processed_count = 0
total_texts = len(df)

print("Zpracovávám texty...")
for idx, text in enumerate(df[text_column]):
    if idx % 10000 == 0:
        print(f"Zpracováno: {idx}/{total_texts} ({idx/total_texts*100:.1f}%)")
    
    amounts = find_dollar_amounts_advanced(str(text))
    all_amounts.extend(amounts)
    processed_count += 1

print(f"\nZpracování dokončeno!")
print(f"Celkem zpracováno textů: {processed_count}")
print(f"Celkem nalezeno dolarových částek: {len(all_amounts)}")

# Statistiky
if all_amounts:
    total_sum = sum(all_amounts)
    print(f"\n{'='*50}")
    print(f"VÝSLEDKY ANALÝZY")
    print(f"{'='*50}")
    print(f"Celková suma všech dolarových částek: ${total_sum:,.2f}")
    print(f"Počet nalezených částek: {len(all_amounts)}")
    print(f"Průměrná částka: ${np.mean(all_amounts):,.2f}")
    print(f"Medián částek: ${np.median(all_amounts):,.2f}")
    print(f"Minimální částka: ${min(all_amounts):,.2f}")
    print(f"Maximální částka: ${max(all_amounts):,.2f}")
    
    # Histogram nejčastějších částek
    print(f"\nNejčastější částky (top 10):")
    from collections import Counter
    amount_counts = Counter(all_amounts)
    for amount, count in amount_counts.most_common(10):
        print(f"${amount:,.2f}: {count}x")
    
    # Rozdělení podle velikosti
    small_amounts = [a for a in all_amounts if a < 100]
    medium_amounts = [a for a in all_amounts if 100 <= a < 1000]
    large_amounts = [a for a in all_amounts if a >= 1000]
    
    print(f"\nRozdělení podle velikosti:")
    print(f"Malé částky (< $100): {len(small_amounts)} částek, suma: ${sum(small_amounts):,.2f}")
    print(f"Střední částky ($100-$999): {len(medium_amounts)} částek, suma: ${sum(medium_amounts):,.2f}")
    print(f"Velké částky (≥ $1000): {len(large_amounts)} částek, suma: ${sum(large_amounts):,.2f}")
    
else:
    print("Nebyly nalezeny žádné dolarové částky v textech.")

# Ukázka některých nalezených částek pro kontrolu
print(f"\nPrvních 20 nalezených částek (pro kontrolu):")
for i, amount in enumerate(all_amounts[:20]):
    print(f"{i+1:2d}. ${amount:,.2f}")

# Export výsledků
print(f"\nUkládání výsledků...")
results_df = pd.DataFrame({
    'amount': all_amounts,
    'amount_formatted': [f"${amount:,.2f}" for amount in all_amounts]
})

# Uložení do CSV
results_df.to_csv('extracted_dollar_amounts.csv', index=False)
print(f"Výsledky uloženy do souboru 'extracted_dollar_amounts.csv'")

# Shrnutí pro rychlý přehled
print(f"\n{'='*60}")
print(f"FINÁLNÍ VÝSLEDEK")
print(f"{'='*60}")
if all_amounts:
    print(f"CELKOVÁ SUMA VŠECH DOLAROVÝCH ČÁSTEK: ${sum(all_amounts):,.2f}")
else:
    print("NEBYLY NALEZENY ŽÁDNÉ DOLAROVÉ ČÁSTKY")
print(f"{'='*60}")

Načítání datasetu...
Dataset úspěšně načten. Počet řádků: 681284
Sloupce: ['id', 'gender', 'age', 'topic', 'sign', 'date', 'text']

Základní informace o datasetu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      681284 non-null  int64 
 1   gender  681284 non-null  object
 2   age     681284 non-null  int64 
 3   topic   681284 non-null  object
 4   sign    681284 non-null  object
 5   date    681284 non-null  object
 6   text    681284 non-null  object
dtypes: int64(2), object(5)
memory usage: 36.4+ MB
None

Prvních 5 řádků:
        id gender  age              topic      sign          date  \
0  2059027   male   15            Student       Leo   14,May,2004   
1  2059027   male   15            Student       Leo   13,May,2004   
2  2059027   male   15            Student       Leo   12,May,2004   
3  2059027   male   15            Student   

In [8]:
import pandas as pd

df = pd.read_csv('./extracted_dollar_amounts.csv')
column_sum = df['amount'].sum()
print(column_sum)

1.0639983289991538e+17
